In [2]:
import urllib
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from os.path import isfile

import torch
from torchvision import transforms
import torchvision.transforms.functional as TF

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import torch.nn.functional as F
from torch import nn

import albumentations as A
from albumentations.pytorch import transforms

import numpy as np
from PIL import Image
import imageio
from matplotlib import pyplot
from tqdm.auto import tqdm
import time

%matplotlib inline

In [3]:
#전역변수 설정

url = "https://www.cs.toronto.edu/~vmnih/data/"

train_input_url = "mass_roads/train/sat/index.html"
train_target_url = "mass_roads/train/map/index.html"
val_input_url = "mass_roads/valid/sat/index.html"
val_target_url = "mass_roads/valid/map/index.html"
test_input_url = "mass_roads/test/sat/index.html"
test_target_url = "mass_roads/test/map/index.html"

train_directory = "C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2.Roadextraction/datasets/train/input"
target_directory = "C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2.Roadextraction/datasets/train/target"
val_train_directory = "C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2.Roadextraction/datasets/validation/input"
val_target_directory = "C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2.Roadextraction/datasets/validation/target"
test_train_directory = "C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2.Roadextraction/datasets/test/input_images"
test_target_directory = "C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2.Roadextraction/datasets/test/target_maps"

dir_train = 'C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2.Roadextraction/datasets/train/input_filtered'
dir_target = 'C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2.Roadextraction/datasets/train/target_filtered'
val_train = 'C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2.Roadextraction/datasets/validation/input_filtered'
val_target = 'C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2.Roadextraction/datasets/validation/target_filtered'
test_train = 'C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2.Roadextraction/datasets/test/input_images_filtered'
test_target = 'C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2.Roadextraction/datasets/test/target_maps_filtered'

In [4]:
#DataSet 설정 및 Trasform 과정
#train 파일(R,G,B)을 img1에 저장 mask(binary)를 img2에 저장 함
#두 파일을 합쳐서 4차원으로 stacking 하고 img3로 지정, normalization 및 crop 진행
#crop 진행 후, img3를 다시 분할해서 img1과 img2로 나눔

class CustomDataset(Dataset): 
    def __init__(self, train_paths, target_paths, transform=None): 
        self.train_paths = [f for f in os.listdir(train_paths) if isfile(train_paths + '/'+ f)] 
        self.target_paths = [f for f in os.listdir(target_paths) if isfile(target_paths + '/' + f)]
        self.a = train_paths
        self.b = target_paths
        self.transform = transform
        
    def __getitem__(self, index): 
        train = np.array(Image.open(self.a+'/'+self.train_paths[index]))
        target = np.array(Image.open(self.b+'/'+self.target_paths[index]))
        
        if self.transform:
            #transforms.Compose
            data = self.transform(image=train, mask=target)
            data_img = data["image"].float()
            data_lab = data["mask"].float()
            data_lab = data_lab.view([1,256,256])/255
            
            data = {'train': data_img, 'target': data_lab}
        return data
        
    def __len__(self):
        return len(self.train_paths)
    
trans = A.Compose([
    A.RandomResizedCrop(256,256),
    A.HorizontalFlip(),
    A.VerticalFlip(),
    A.Normalize(mean=0.5, std=0.5),
    transforms.ToTensorV2(transpose_mask=True)
])



In [5]:
#Unet 모델 쌓기

class UNet(nn.Module):  
    def __init__(self): 
# super(subclass, self) : subclass에서 base class의 내용을 오버라이드해서 사용하고 싶을 때
        super(UNet, self).__init__() 
        
# 네트워크에서 반복적으로 사용되는 Conv + BatchNorm + Relu를 합쳐서 하나의 함수로 정의
       	def CBR2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True):
            layers = []
            layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                 kernel_size=kernel_size, stride=stride, padding=padding,
                                 bias=bias)]
            layers += [nn.BatchNorm2d(num_features=out_channels)]
            layers += [nn.ReLU()]

            cbr = nn.Sequential(*layers) # *으로 list unpacking 

            return cbr

        # Contracting path
        self.enc1_1 = CBR2d(in_channels=3, out_channels=64)
        self.enc1_2 = CBR2d(in_channels=64, out_channels=64)
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.enc2_1 = CBR2d(in_channels=64, out_channels=128)
        self.enc2_2 = CBR2d(in_channels=128, out_channels=128)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        
        self.enc3_1 = CBR2d(in_channels=128, out_channels=256)
        self.enc3_2 = CBR2d(in_channels=256, out_channels=256)
        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.enc4_1 = CBR2d(in_channels=256, out_channels=512)
        self.enc4_2 = CBR2d(in_channels=512, out_channels=512)
        self.pool4 = nn.MaxPool2d(kernel_size=2)
        
        self.enc5_1 = CBR2d(in_channels=512, out_channels=1024)
        # Expansive path
        self.dec5_1 = CBR2d(in_channels=1024, out_channels=512)

        self.unpool4 = nn.ConvTranspose2d(in_channels=512, out_channels=512,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec4_2 = CBR2d(in_channels=2 * 512, out_channels=512)
        self.dec4_1 = CBR2d(in_channels=512, out_channels=256)
        self.unpool3 = nn.ConvTranspose2d(in_channels=256, out_channels=256,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec3_2 = CBR2d(in_channels=2 * 256, out_channels=256)
        self.dec3_1 = CBR2d(in_channels=256, out_channels=128)
        self.unpool2 = nn.ConvTranspose2d(in_channels=128, out_channels=128,
                                          kernel_size=2, stride=2, padding=0, bias=True)
        
        self.dec2_2 = CBR2d(in_channels=2 * 128, out_channels=128)
        self.dec2_1 = CBR2d(in_channels=128, out_channels=64)
        self.unpool1 = nn.ConvTranspose2d(in_channels=64, out_channels=64,
                                          kernel_size=2, stride=2, padding=0, bias=True)
        
        self.dec1_2 = CBR2d(in_channels=2 * 64, out_channels=64)
        self.dec1_1 = CBR2d(in_channels=64, out_channels=64)
        self.fc = nn.Conv2d(in_channels=64, out_channels=1, kernel_size=1, stride=1, padding=0, bias=True)
        
# __init__ 함수에서 선언한 layer들 연결해서 data propa flow 만들기
    def forward(self, x):
        enc1_1 = self.enc1_1(x)
        enc1_2 = self.enc1_2(enc1_1)
        pool1 = self.pool1(enc1_2)
        
        enc2_1 = self.enc2_1(pool1)
        enc2_2 = self.enc2_2(enc2_1)
        pool2 = self.pool2(enc2_2)
        
        enc3_1 = self.enc3_1(pool2)
        enc3_2 = self.enc3_2(enc3_1)
        pool3 = self.pool3(enc3_2)
        
        enc4_1 = self.enc4_1(pool3)
        enc4_2 = self.enc4_2(enc4_1)
        pool4 = self.pool4(enc4_2)
        
        enc5_1 = self.enc5_1(pool4)
        dec5_1 = self.dec5_1(enc5_1)
        
        unpool4 = self.unpool4(dec5_1)
        cat4 = torch.cat((unpool4, enc4_2), dim=1)
        dec4_2 = self.dec4_2(cat4)
        dec4_1 = self.dec4_1(dec4_2)
        
        unpool3 = self.unpool3(dec4_1)
        cat3 = torch.cat((unpool3, enc3_2), dim=1)
        dec3_2 = self.dec3_2(cat3)
        dec3_1 = self.dec3_1(dec3_2)

        unpool2 = self.unpool2(dec3_1)
        cat2 = torch.cat((unpool2, enc2_2), dim=1)
        dec2_2 = self.dec2_2(cat2)
        dec2_1 = self.dec2_1(dec2_2)

        unpool1 = self.unpool1(dec2_1)
        cat1 = torch.cat((unpool1, enc1_2), dim=1)
        #tensor 합치기: Unet에서 Copy and Crop 과정을 담당한다
        dec1_2 = self.dec1_2(cat1)
        dec1_1 = self.dec1_1(dec1_2)

        out = self.fc(dec1_1)

        return out # data가 모든 layer를 거쳐서 나온 output 값

In [7]:
#트레이닝
## 하이퍼 파라미터 설정

lr = 1e-3
batch_size = 8
num_epoch = 100

ckpt_dir = 'C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2.Roadextraction/checkpntlog/checkpoint'
log_dir = 'C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2.Roadextraction/checkpntlog/log'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# transform 적용해서 데이터 셋 불러오기
dataset_train = CustomDataset(dir_train, dir_target, transform=trans)
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

# val set도 동일하게 진행
dataset_val = CustomDataset(val_train, val_target, transform = trans)
loader_val = DataLoader(dataset_val, batch_size=batch_size , shuffle=True)

# 네트워크 불러오기
net = UNet().to(device) # device : cpu or gpu

# loss 정의
fn_loss = nn.BCEWithLogitsLoss().to(device)

# Optimizer 정의
optim = torch.optim.Adam(net.parameters(), lr = lr ) 

# 기타 variables 설정
num_train = len(dataset_train)
num_val = len(dataset_val)


num_train_for_epoch = np.ceil(num_train/batch_size) # np.ceil : 소수점 반올림
num_val_for_epoch = np.ceil(num_val/batch_size)


# 기타 function 설정
fn_tonumpy = lambda x : x.to('cpu').detach().numpy().transpose(0,2,3,1) # device 위에 올라간 텐서를 detach 한 뒤 numpy로 변환
fn_denorm = lambda x, mean, std : (x * std) + mean 
fn_classifier = lambda x :  1.0 * (x > 0.5)  # threshold 0.5 기준으로 indicator function으로 classifier 구현

# Tensorbord
writer_train = SummaryWriter(log_dir=os.path.join(log_dir,'train'))
writer_val = SummaryWriter(log_dir = os.path.join(log_dir,'val'))
writer_overall = SummaryWriter(log_dir = os.path.join(log_dir,'overall'))

In [8]:
# 네트워크 저장하기
# train을 마친 네트워크 저장 
# net : 네트워크 파라미터, optim  두개를 dict 형태로 저장
def save(ckpt_dir,net,optim,epoch):
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    torch.save({'net':net.state_dict(),'optim':optim.state_dict()},'%s/model_epoch%d.pth'%(ckpt_dir,epoch))

# 네트워크 불러오기
def load(ckpt_dir,net,optim):
    if not os.path.exists(ckpt_dir): # 저장된 네트워크가 없다면 인풋을 그대로 반환
        epoch = 0
        return net, optim, epoch
    
    ckpt_lst = os.listdir(ckpt_dir) # ckpt_dir 아래 있는 모든 파일 리스트를 받아온다
    ckpt_lst.sort(key = lambda f : int(''.join(filter(str.isdigit,f))))

    dict_model = torch.load('%s/%s' % (ckpt_dir,ckpt_lst[-1]))
    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])
    epoch = int(ckpt_lst[-1].split('epoch')[1].split('.pth')[0])

    return net,optim,epoch

In [ ]:
# 네트워크 학습시키기
start_epoch = 0
net, optim, start_epoch = load(ckpt_dir = ckpt_dir, net = net, optim = optim) # 저장된 네트워크 불러오기

for epoch in tqdm(range(start_epoch+1,num_epoch +1)):
    net.train()
    loss_arr = []
    for batch, data in enumerate(loader_train,1): # 1은 뭐니 > index start point
        # forward
        label = data['target'].to(device)   # 데이터 device로 올리기     
        inputs = data['train'].to(device)
        output = net(inputs) 

        # backward
        optim.zero_grad()  # gradient 초기화
        loss = fn_loss(output, label)  # output과 label 사이의 loss 계산
        loss.backward() # gradient backpropagation
        optim.step() # backpropa 된 gradient를 이용해서 각 layer의 parameters update

        # save loss
        loss_arr += [loss.item()]

        # tensorbord에 결과값들 저정하기
        label = fn_tonumpy(label)
        inputs = fn_tonumpy(fn_denorm(inputs,0.5,0.5))
        output = fn_tonumpy(fn_classifier(output))

        writer_train.add_image('label', label, num_train_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
        writer_train.add_image('input', inputs, num_train_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
        writer_train.add_image('output', output, num_train_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
        writer_overall.add_image('label_train', label, num_train_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
        writer_overall.add_image('input_train', inputs, num_train_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
        writer_overall.add_image('output_train', output, num_train_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
    
    writer_train.add_scalar('loss', np.mean(loss_arr), epoch)
    writer_overall.add_scalar('loss_train', np.mean(loss_arr), epoch)
    
    # validation
    with torch.no_grad(): # validation 이기 때문에 backpropa 진행 x, 학습된 네트워크가 정답과 얼마나 가까운지 loss만 계산
        net.eval() # 네트워크를 evaluation 용으로 선언
        loss_arr = []

        for batch, data in enumerate(loader_val,1):
            # forward
            label = data['target'].to(device)
            inputs = data['train'].to(device)
            output = net(inputs)

            # loss 
            loss = fn_loss(output,label)
            loss_arr += [loss.item()]
            print('valid : epoch %04d / %04d | Batch %04d \ %04d | Loss %04f'%(epoch,num_epoch,batch,num_val_for_epoch,np.mean(loss_arr)))

            # Tensorboard 저장하기
            label = fn_tonumpy(label)
            inputs = fn_tonumpy(fn_denorm(inputs, mean=0.5, std=0.5))
            output = fn_tonumpy(fn_classifier(output))

            writer_val.add_image('label', label, num_val_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
            writer_val.add_image('input', inputs, num_val_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
            writer_val.add_image('output', output, num_val_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
            writer_overall.add_image('label_val', label, num_val_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
            writer_overall.add_image('input_val', inputs, num_val_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
            writer_overall.add_image('output_val', output, num_val_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
            
        writer_val.add_scalar('loss', np.mean(loss_arr), epoch)
        writer_overall.add_scalar('loss_val', np.mean(loss_arr), epoch)
        
        # epoch이 끝날때 마다 네트워크 저장
        save(ckpt_dir=ckpt_dir, net = net, optim = optim, epoch = epoch)
    time.sleep(0.1)
    
writer_train.close()
writer_val.close()
writer_overall.close()

  0%|          | 0/100 [00:00<?, ?it/s]

valid : epoch 0001 / 0100 | Batch 0001 \ 0002 | Loss 0.245964
valid : epoch 0001 / 0100 | Batch 0002 \ 0002 | Loss 0.241043
valid : epoch 0002 / 0100 | Batch 0001 \ 0002 | Loss 0.173935
valid : epoch 0002 / 0100 | Batch 0002 \ 0002 | Loss 0.200217
valid : epoch 0003 / 0100 | Batch 0001 \ 0002 | Loss 0.160989
valid : epoch 0003 / 0100 | Batch 0002 \ 0002 | Loss 0.196467
valid : epoch 0004 / 0100 | Batch 0001 \ 0002 | Loss 0.184306
valid : epoch 0004 / 0100 | Batch 0002 \ 0002 | Loss 0.165440
valid : epoch 0005 / 0100 | Batch 0001 \ 0002 | Loss 0.194291
valid : epoch 0005 / 0100 | Batch 0002 \ 0002 | Loss 0.141803
valid : epoch 0006 / 0100 | Batch 0001 \ 0002 | Loss 0.120479
valid : epoch 0006 / 0100 | Batch 0002 \ 0002 | Loss 0.149474
valid : epoch 0007 / 0100 | Batch 0001 \ 0002 | Loss 0.159395
valid : epoch 0007 / 0100 | Batch 0002 \ 0002 | Loss 0.168555
valid : epoch 0008 / 0100 | Batch 0001 \ 0002 | Loss 0.151750
valid : epoch 0008 / 0100 | Batch 0002 \ 0002 | Loss 0.154649
valid : 

In [10]:
dataset_test = CustomDataset(test_train, test_target, transform=trans)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
writer_test = SummaryWriter(log_dir = os.path.join(log_dir,'test'))
num_test = len(dataset_test)
num_test_for_epoch = np.ceil(num_test/batch_size)



#학습 완료된 모델을 불러온 후, 모델에 test dataset을 적용한다
net, optim, epoch = load(ckpt_dir = ckpt_dir, net = net, optim = optim)
with torch.no_grad(): #test 이기 때문에 backpropa 진행 x, 학습된 네트워크가 정답과 얼마나 가까운지 loss만 계산
    net.eval() # 네트워크를 evaluation 용으로 선언
    loss_arr = []
    for batch, data in tqdm(enumerate(loader_test,1)):
        # forward
        label = data['target'].to(device)
        inputs = data['train'].to(device)
        output = net(inputs)

        # loss 
        loss = fn_loss(output,label)
        loss_arr += [loss.item()]
        print('valid : epoch %04d / %04d | Batch %04d \ %04d | Loss %04f'%(epoch,num_epoch,batch,num_val_for_epoch,np.mean(loss_arr)))

        # Tensorboard 저장하기
        label = fn_tonumpy(label)
        inputs = fn_tonumpy(fn_denorm(inputs, mean=0.5, std=0.5))
        output = fn_tonumpy(fn_classifier(output))
        writer_test.add_image('label', label, num_val_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
        writer_test.add_image('input', inputs, num_val_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
        writer_test.add_image('output', output, num_val_for_epoch * (epoch - 1) + batch, dataformats='NHWC')        
        writer_test.add_scalar('loss_val', np.mean(loss_arr), epoch)
        
        time.sleep(0.1)
        
        
writer_test.close()

0it [00:00, ?it/s]

valid : epoch 0100 / 0100 | Batch 0001 \ 0002 | Loss 0.050400
valid : epoch 0100 / 0100 | Batch 0002 \ 0002 | Loss 0.057731
valid : epoch 0100 / 0100 | Batch 0003 \ 0002 | Loss 0.057828
valid : epoch 0100 / 0100 | Batch 0004 \ 0002 | Loss 0.065320
valid : epoch 0100 / 0100 | Batch 0005 \ 0002 | Loss 0.069897
valid : epoch 0100 / 0100 | Batch 0006 \ 0002 | Loss 0.068343
valid : epoch 0100 / 0100 | Batch 0007 \ 0002 | Loss 0.076026


In [ ]:
그래서 지금 해야 되는 것은 여기에 정확도를 평가하는 지표를 찾아내야 하는 것인데,
이것이 어떻게 가능한지에 대해서 논문을 찾아볼 필요가 있을 듯
또한 딥러닝을 이용해서 공간정보를 어떻게 이용할 지에 대해서도 곰곰히 생각해 보아야 할 문제이다


In [13]:
#이미지 stacking
a = os.listdir(dir_train)
img1 = np.array(Image.open(dir_train+'/'+a[0]))
img2 = np.array(Image.open(dir_target+'/'+a[0]))

img3 = np.dstack([img1,img2])
print(img3[:,:,0:3].shape)
img3[:,:,3:4]


(1500, 1500, 3)


array([[[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       ...,

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]]], dtype=uint8)

In [8]:
#Tensor을 Image로 출력하는 함수
tf = transforms.ToPILImage()
img_t = tf(dataset.__getitem__(0)[1])
img_t.show()


AttributeError: module 'albumentations.pytorch.transforms' has no attribute 'ToPILImage'

In [35]:
#Tensor을 Image로 출력하는 함수
tf = transforms.ToPILImage()
img_t = tf(images1[0])
img_t.show()

In [9]:
a,b = dataset_train.__getitem__(0)


torch.Size([1, 256, 256])


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [8]:
#datloader 설정
#train은 images1에 mask는 images2에 저장
dataset = CustomDataset(dir_train, dir_target, transform=trans)
data = dataset.__getitem__(0)

train_dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

batch_iterator = iter(train_dataloader)
images1, images2 = next(batch_iterator)

In [10]:
from torchsummary import summary
model = UNet()
summary(model.cuda(),input_size=(3,256,256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
            Conv2d-4         [-1, 64, 256, 256]          36,928
       BatchNorm2d-5         [-1, 64, 256, 256]             128
              ReLU-6         [-1, 64, 256, 256]               0
         MaxPool2d-7         [-1, 64, 128, 128]               0
            Conv2d-8        [-1, 128, 128, 128]          73,856
       BatchNorm2d-9        [-1, 128, 128, 128]             256
             ReLU-10        [-1, 128, 128, 128]               0
           Conv2d-11        [-1, 128, 128, 128]         147,584
      BatchNorm2d-12        [-1, 128, 128, 128]             256
             ReLU-13        [-1, 128, 128, 128]               0
        MaxPool2d-14          [-1, 128,

In [11]:
a = torch.tensor([[[[ 1,  2,  3,  4],
                    [ 5,  6,  7,  8],
                    [ 9, 10, 11, 12],
                    [13, 14, 15, 16]],
                  [[ 1,  2,  3,  4],
                    [ 5,  6,  7,  8],
                    [ 9, 10, 11, 12],
                    [13, 14, 15, 16]], 
                  [[ 1,  2,  3,  4],
                    [ 5,  6,  7,  8],
                    [ 9, 10, 11, 12],
                    [13, 14, 15, 16]]],
                 [[[ 1,  2,  3,  4],
                    [ 5,  6,  7,  8],
                    [ 9, 10, 11, 12],
                    [13, 14, 15, 16]],
                 [[ 1,  2,  3,  4],
                    [ 5,  6,  7,  8],
                    [ 9, 10, 11, 12],
                    [13, 14, 15, 16]],
                 [[ 1,  2,  3,  4],
                    [ 5,  6,  7,  8],
                    [ 9, 10, 11, 12],
                    [13, 14, 15, 16]]],
                 [[[ 1,  2,  3,  4],
                    [ 5,  6,  7,  8],
                    [ 9, 10, 11, 12],
                    [13, 14, 15, 16]],
                 [[ 1,  2,  3,  4],
                    [ 5,  6,  7,  8],
                    [ 9, 10, 11, 12],
                    [13, 14, 15, 16]],
                 [[ 1,  2,  3,  4],
                    [ 5,  6,  7,  8],
                    [ 9, 10, 11, 12],
                    [13, 14, 15, 16]]]]).float()

b = nn.ConvTranspose2d(in_channels=3, out_channels=3, kernel_size=2, stride=2, padding=0, bias=True)
torch.cat((a,a),dim=1)

AttributeError: 